# Read Your Key and Set Env

In [ ]:
# first step everytime you need to run

import json
import os
# Upload your spark_nlp_for_healthcare.json  to the default directory and then run this cell to authenticate 
with open('4.0.0.spark_nlp_for_healthcare.json', 'r') as f:
    for k, v in json.load(f).items():
        %set_env $k=$v

%set_env PYSPARK=3.2.2
%set_env SPARK_HOME=/home/ec2-user/SageMaker/spark-3.2.2-bin-hadoop2.7


with open('4.0.0.spark_nlp_for_healthcare.json') as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)


!java -version

# Dowload and Install OCR

In [2]:
# second step only run when intalling or reinstalling the library 
# No need to run it over and over

!wget -O https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jsl_sagemaker_setup_with_OCR.sh -O jsl_sagemaker_setup_with_OCR.sh
!bash jsl_sagemaker_setup_with_OCR.sh

wget: missing URL
Usage: wget [OPTION]... [URL]...

Try `wget --help' for more options.
setup SageMaker for PySpark 3.2.2 and Spark NLP 4.0.0 and Spark OCR 4.0.0 and Spark NLP for Healthcare 4.0.0
alternatives version 1.7.4 - Copyright (C) 2001 Red Hat, Inc.
This may be freely redistributed under the terms of the GNU Public License.

usage: alternatives --install <link> <name> <path> <priority>
                    [--initscript <service>]
                    [--family <family>]
                    [--slave <link> <name> <path>]*
       alternatives --remove <name> <path>
       alternatives --auto <name>
       alternatives --config <name>
       alternatives --display <name>
       alternatives --set <name> <path>
       alternatives --list

common options: --verbose --test --help --usage --version --keep-missing
                --altdir <directory> --admindir <directory>
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This be

# Start Your Sessions

In [4]:
import sys, os
import base64
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import sparkocr
from sparkocr import start
from sparkocr.enums import *
from sparkocr.utils import *
from sparkocr.metrics import score
from sparkocr.transformers import *

from pyspark.ml import PipelineModel
from pyspark.sql import functions as F
from pyspark.sql import SparkSession

import pkg_resources

# Start spark
spark = sparkocr.start(secret=SPARK_OCR_SECRET, 
                       nlp_version=PUBLIC_VERSION
                       )

spark.sparkContext.setLogLevel("ERROR")

# check that spark started
spark

Spark version: 3.2.1
Spark NLP version: 4.0.0
Spark NLP for Healthcare version: 4.0.0
Spark OCR version: 4.0.0



22/07/25 20:38:00 INFO LicenseValidator$: Using legacy license, can not register for credential notifications.
22/07/25 20:38:00 INFO LicenseValidator$: Using legacy license, cannot start periodic credential refresh.


# Create Pipeline

In [6]:
!wget -q -O sample_doc.pdf http://www.asx.com.au/asxpdf/20171103/pdf/43nyyw9r820c6r.pdf

In [7]:
# Transform PDF document to images per page
pdf_to_image = PdfToImage()\
      .setInputCol("content")\
      .setOutputCol("image")
# Run OCR
ocr = ImageToText()\
      .setInputCol("image")\
      .setOutputCol("text")\
      .setConfidenceThreshold(65)
      # .setKeepLayout(True) \ # to preserve the layout of the input
pdf_to_text_pipeline = PipelineModel(stages=[
    pdf_to_image,
    ocr
])
pdf = 'sample_doc.pdf'
pdf_example_df = spark.read.format("binaryFile").load(pdf).cache()
result = pdf_to_text_pipeline.transform(pdf_example_df).cache()
result.select("pagenum","text", "confidence").show()

+-------+--------------------+-----------------+
|pagenum|                text|       confidence|
+-------+--------------------+-----------------+
|      0|ASX ANNOUNCEMENT\...|90.42361619737413|
+-------+--------------------+-----------------+

